In [71]:
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
import math
from tqdm.notebook import tqdm
import time
import pyautogui

In [62]:
image = cv2.imread('./flp/traintry001/r1249.jpg')

In [63]:
# 삼분할법에 대한 점수계산 코드

#각 이미지에 대한 크기 반환
h, w, c = image.shape
print('width:  ', w)
print('height: ', h)
print('channel:', c)

width:   275
height:  183
channel: 3


In [64]:
#method1 

#삼분할관련 교차지점 point 반환 ->p1,p2,p3,p4

x1, x2 = w//3, w*2//3
y1, y2 = h//3, h*2//3

print("x1, x2: {}, {}".format(x1, x2))
print("y1, y2: {}, {}".format(y1, y2))

p1 = [x1, y1]
p2 = [x1, y2]
p3 = [x2, y1]
p4 = [x2, y2]

x1, x2: 91, 183
y1, y2: 61, 122


In [65]:
def cal_d(p,n):
    d = (p[0]-n[0])**2 + (p[1]-n[1])**2
    return d**(1/2) 

def cal_p(x,y):
    return (x-y)

In [91]:
#사진상 사람의 코 위치 검출
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

# For static images:
with mp_holistic.Holistic(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    refine_face_landmarks=True) as holistic:
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.pose_landmarks:
        pnx = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * w
        pny = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * h
        pn = [pnx, pny]
        #pn-> 코의 위치
    
    plist = [p1, p2, p3, p4]
    #pnd = point to nose distance
    pnd= min(cal_d(p1, pn), cal_d(p2, pn), cal_d(p3, pn), cal_d(p4, pn))
    print(pnd)
    r = cal_d([0,0], p1)
    
    score1 = (r - pnd)/(r)
    print(score1)

35.20317548717855
0.66810127858288


In [97]:
#method 2

#주요면적 크기

BG_COLOR = (255, 255, 255) # gray
MASK_COLOR = (1, 1, 1) # white
annotated_image = image.copy()

condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
bg_image = np.zeros(image.shape, dtype=np.uint8)
bg_image[:] = BG_COLOR
annotated_image = np.where(condition, annotated_image, bg_image)

cv2.imshow("annotated_image", annotated_image)
cv2.imshow('image', image)
cv2.waitKey()

cv2.destroyAllWindows()

In [95]:
count000=0
for i in annotated_image:
    ic0=i.tolist()
    count000 += ic0.count([255,255,255])
print(count000)

43702


In [96]:
print(count000/(w*h))
portion = 1- count000/(w*h)

minp = min(abs(0.1-portion), abs(0.56-portion), abs(0.82-portion))
print(minp)

if portion<0.82 and portion>0.1:
    a=1/0.23
    score2 = a*(0.23-minp)
    
elif portion>=0.82:
    a=1/0.18
    score2 = a*(0.18-minp)

elif portion<=0.1:
    a=1/0.1
    score2 = a*(0.1-minp)
    
print(score2)

print((score1+score2)/2)

0.8632493827160493
0.036750617283950654
0.8402147074610841
0.754157993021982


In [80]:
import glob
import os

In [81]:
path = './flp/traintry001/'
file_list = os.listdir(path)
jpg_file_list = [file for file in file_list if file.endswith(".jpg")]
jpg_file_list

['1.jpg',
 '10.jpg',
 '101.jpg',
 '102.jpg',
 '105.jpg',
 '106.jpg',
 '107.jpg',
 '108.jpg',
 '109.jpg',
 '11.jpg',
 '112.jpg',
 '113.jpg',
 '115.jpg',
 '117.jpg',
 '118.jpg',
 '119.jpg',
 '12.jpg',
 '120.jpg',
 '121.jpg',
 '125.jpg',
 '126.jpg',
 '127.jpg',
 '128.jpg',
 '129.jpg',
 '13.jpg',
 '131.jpg',
 '135.jpg',
 '136.jpg',
 '138.jpg',
 '139.jpg',
 '14.jpg',
 '140.jpg',
 '141.jpg',
 '142.jpg',
 '143.jpg',
 '144.jpg',
 '145.jpg',
 '147.jpg',
 '148.jpg',
 '149.jpg',
 '15.jpg',
 '150.jpg',
 '151.jpg',
 '152.jpg',
 '153.jpg',
 '154.jpg',
 '155.jpg',
 '156.jpg',
 '158.jpg',
 '159.jpg',
 '16.jpg',
 '160.jpg',
 '161.jpg',
 '162.jpg',
 '163.jpg',
 '164.jpg',
 '165.jpg',
 '166.jpg',
 '167.jpg',
 '168.jpg',
 '169.jpg',
 '172.jpg',
 '175.jpg',
 '176.jpg',
 '179.jpg',
 '18.jpg',
 '182.jpg',
 '184.jpg',
 '186.jpg',
 '187.jpg',
 '189.jpg',
 '19.jpg',
 '191.jpg',
 '192.jpg',
 '193.jpg',
 '195.jpg',
 '197.jpg',
 '199.jpg',
 '2.jpg',
 '20.jpg',
 '200.jpg',
 '201.jpg',
 '202.jpg',
 '204.jpg',
 '206.

In [87]:
photo_data = []
IMAGE_FILES = jpg_file_list

def cal_d(p,n):
    d = (p[0]-n[0])**2 + (p[1]-n[1])**2
    return d**(1/2) 

def cal_p(x,y):
    return (x-y)

with mp_holistic.Holistic(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    refine_face_landmarks=True) as holistic:
    idx=0
    for file in tqdm(IMAGE_FILES):
#         image = cv2.imread('./traindata02/'+file)
        image = cv2.imread('./0a.jpg')


########################################################################################################

#score1#
# 삼분할법에 대한 점수계산 코드

        #각 이미지에 대한 크기 반환
        h, w, c = image.shape

        #삼분할관련 교차지점 point 반환 ->p1,p2,p3,p4

        x1, x2 = w//3, w*2//3
        y1, y2 = h//3, h*2//3

        p1 = [x1, y1]
        p2 = [x1, y2]
        p3 = [x2, y1]
        p4 = [x2, y2]
        # Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if results.pose_landmarks:
            pnx = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * w
            pny = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * h
            pn = [pnx, pny]
            #pn-> 코의 위치
    
        plist = [p1, p2, p3, p4]
        #pnd = point to nose distance
        pnd= min(cal_d(p1, pn), cal_d(p2, pn), cal_d(p3, pn), cal_d(p4, pn))
 
        r = cal_d([0,0], p1)
    
        score1 = (r - pnd)/(r)

########################################################################################################        

#score2#

        # Draw segmentation on the image.
        # To improve segmentation around boundaries, consider applying a joint
        # bilateral filter to "results.segmentation_mask" with "image".
        annotated_image = image.copy()
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR
        annotated_image = np.where(condition, annotated_image, bg_image)

        count000=0
        for i in annotated_image:
            ic0=i.tolist()
            count000 += ic0.count([255,255,255])

        portion = 1- count000/(w*h)

        minp = min(abs(0.1-portion), abs(0.56-portion), abs(0.82-portion))

        if portion<0.82 and portion>0.1:
            a=1/0.23
            score2 = a*(0.23-minp)

        elif portion>=0.82:
            a=1/0.18
            score2 = a*(0.18-minp)

        elif portion<=0.1:
            a=1/0.1
            score2 = a*(0.1-minp)

        score = (score1+score2)/2
########################################################################################################

#prepare data#

        n=[]
        for data_point in results.pose_landmarks.landmark:
            n.append(data_point.x)
            n.append(data_point.y)
            n.append(data_point.z)
        n.append(score)

        photo_data.append(n)

########################################################################################################
        

  0%|          | 0/1337 [00:00<?, ?it/s]

In [90]:
len(photo_data)

1337

In [88]:
import csv
f = open('./flp/photo_data001.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerows(photo_data)
f.close()